In [56]:
from rembg import remove

In [58]:
# Load the image
input_path = "public/images/logo1.png"
output_path = "public/images/logo1-removebg.png"

print("Using targeted white background removal to preserve text...")

from collections import deque
import numpy as np
from PIL import Image, ImageFilter

def remove_white_background_carefully(
    input_path: str,
    output_path: str,
    tol: int = 20,  # Higher tolerance for near-white
    feather: float = 0.5,  # Light feathering
) -> None:
    """
    Remove white background while preserving text and handling circular backgrounds.
    """
    # Open and ensure RGBA
    img = Image.open(input_path).convert("RGBA")
    arr = np.array(img)  # HxWx4, uint8
    
    # Split channels
    R, G, B, A = arr[..., 0], arr[..., 1], arr[..., 2], arr[..., 3]
    
    # Define white-ish pixels (more aggressive to catch near-white)
    whiteish = (
        (R >= 255 - tol) & (G >= 255 - tol) & (B >= 255 - tol) |
        # Also catch very light grays
        ((R == G) & (G == B) & (R >= 235))
    )
    
    h, w = whiteish.shape
    visited = np.zeros_like(whiteish, dtype=bool)
    q = deque()
    
    def try_enqueue(y, x):
        if (0 <= y < h) and (0 <= x < w) and (not visited[y, x]) and whiteish[y, x]:
            visited[y, x] = True
            q.append((y, x))
    
    # Seed the BFS with all border pixels that are white-ish
    for x in range(w):
        if whiteish[0, x]:
            visited[0, x] = True
            q.append((0, x))
        if whiteish[h - 1, x]:
            visited[h - 1, x] = True
            q.append((h - 1, x))
    for y in range(h):
        if whiteish[y, 0]:
            visited[y, 0] = True
            q.append((y, 0))
        if whiteish[y, w - 1]:
            visited[y, w - 1] = True
            q.append((y, w - 1))
    
    # 4-connected BFS to find all edge-connected white background
    while q:
        y, x = q.popleft()
        if y > 0:
            try_enqueue(y - 1, x)
        if y + 1 < h:
            try_enqueue(y + 1, x)
        if x > 0:
            try_enqueue(y, x - 1)
        if x + 1 < w:
            try_enqueue(y, x + 1)
    
    # visited == background to erase
    bg_mask = visited
    
    # Additional processing: Remove white areas inside the circle
    # Find the center and radius of the circular logo area
    center_y, center_x = h // 2, w // 2
    radius = min(h, w) // 3  # Approximate radius
    
    # Create a circular mask
    y_coords, x_coords = np.ogrid[:h, :w]
    circular_mask = (x_coords - center_x)**2 + (y_coords - center_y)**2 <= radius**2
    
    # Inside the circle, also remove white areas
    inside_circle_white = whiteish & circular_mask
    bg_mask = bg_mask | inside_circle_white
    
    # Build new alpha: make bg transparent (0), keep original alpha elsewhere
    new_alpha = np.where(bg_mask, 0, A)
    
    # Apply light feathering for smoother edges
    if feather and feather > 0:
        alpha_img = Image.fromarray(new_alpha, mode="L").filter(
            ImageFilter.GaussianBlur(radius=float(feather))
        )
        new_alpha = np.array(alpha_img, dtype=np.uint8)
    
    # Apply alpha
    out_arr = arr.copy()
    out_arr[..., 3] = new_alpha
    
    # Save as PNG to preserve transparency
    out_img = Image.fromarray(out_arr, mode="RGBA")
    out_img.save(output_path, optimize=True)
    print(f"Careful white background removal completed: {output_path}")

# Call the careful function
remove_white_background_carefully(input_path, output_path, tol=20, feather=0.5)



Using targeted white background removal to preserve text...
Careful white background removal completed: public/images/logo1-removebg.png


In [59]:
from PIL import Image

# Open the background removed image
img = Image.open(output_path)

# Set new size (you can adjust this as needed)
new_size = (2000, 2000)  # Increased from 100x100 for better readability

# Resize the image with high quality
resized_img = img.resize(new_size, Image.Resampling.LANCZOS)

# Save the resized image
resized_img.save(output_path, optimize=True)

print(f"Image resized and saved to {output_path}")
print(f"Final size: {new_size[0]}x{new_size[1]} pixels")



Image resized and saved to public/images/logo1-removebg.png
Final size: 2000x2000 pixels
